# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> IMT1001 - Introducción a Ingeniería Matemática
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>

# Taller 4: Webscraping.

El proceso de webscrapping en su expresión mas básica consiste en la extracción de datos desde el texto plano que estructura la página web (HTML). Requests nos brinda herramientas para la extracción de la página, y a continuación podemos usar otras librerías como BeautifulSoup para la extracción de elementos o información contenida en el texto plano.

Este Notebook contiene los códigos de ejemplo presentados en la videoclase titulada *"Raspado de páginas web - webscraping"*.


In [2]:
#Importación de librerías
import requests
from bs4 import BeautifulSoup 
import pandas as pd

# 2.Código HTML

Veamos primero un ejemplo simple de código HTML para comprender su estructura general.

Se sugiere descargar el archivo HTML de ejemplo y abrirlo con un navegador (Firefox, Chrome, etc.), de manera de visualizarlo como pagina web.

In [4]:
#archivo HTML de ejemplo disponible en Github
html_text=requests.get('https://raw.githubusercontent.com/paguirre-uc/imt1001/main/ejemplo.html').content

#alternativa si descargan el archivo HTML de ejemplo a su directorio local.
#html_text=open('ejemplo.html','r').read()

html_text

b"<html>\n    <head>\n        <title>IMT1001 Taller 4: Web Scrapping</title>\n    </head>\n    <body>\n        <h1>1. C\xc3\xb3digo HTML</h1>\n        <p> Este es un ejemplo para comprender la estructura de un documento HTML. Los pasos a seguir para extrare los datos de inter\xc3\xa9s son:\n          <ul id='list' class='pasos'>\n            <li> Leer el c\xc3\xb3digo HTML</li>\n            <li> Formatear (parse) usando BeautifulSoup, y </li>\n            <li> Extraer la informaci\xc3\xb3n de inter\xc3\xa9s. </li>\n          </ul>\n        </p>\n\n        <p>Por ejemplo, podemos extraer p\xc3\xa1rrafos de texto, los cuales pueden ser extensos, o muy breves. Como comentamos en clase, los cuerpos de texto que se extraen de la web pueden ser utilizados por ejemplo para construir grandes datasets de entrenamiento de modelos de lenguaje. Esto es una excelente motivaci\xc3\xb3n para aprender a hacer webscraping!!</p>\n       \n       <p>   </p>\n       <p>Tambi\xc3\xa9n podemos crear una tab

In [5]:
from bs4 import BeautifulSoup 

#Creamos una "sopa" a partir del códiog HTML
soup = BeautifulSoup(html_text)

In [12]:
#busquemos todos los párrafos
paragraphs=soup.find_all('p')

In [15]:
#el método find devuelve el primer elemento del tipo solicitado
soup.find('p')

<p> Este es un ejemplo para comprender la estructura de un documento HTML. Los pasos a seguir para extrare los datos de interés son:
          </p>

In [13]:
#para cada párrafo podemos extraer el texto sin etiquetas
for p in paragraphs:
  print(p.text)  

 Este es un ejemplo para comprender la estructura de un documento HTML. Los pasos a seguir para extrare los datos de interés son:
          
Por ejemplo, podemos extraer párrafos de texto, los cuales pueden ser extensos, o muy breves. Como comentamos en clase, los cuerpos de texto que se extraen de la web pueden ser utilizados por ejemplo para construir grandes datasets de entrenamiento de modelos de lenguaje. Esto es una excelente motivación para aprender a hacer webscraping!!
 
También podemos crear una tabla:
            
 


In [6]:
#accedemos a la tabla de datos. Esto devuelve la primera tabla de datos del código.
soup.table

<table>
<tr>
<th>Curso</th>
<th>Créditos</th>
<th>N Estudiantes</th>
</tr>
<tr>
<td>IMT1101</td>
<td>10</td>
<td>42</td>
</tr>
<tr>
<td>OFG1100</td>
<td>5</td>
<td>25</td>
</tr>
</table>

In [7]:
#Otra forma de extraer la tabla de datos.
table=soup.find('table')
print(table)

<table>
<tr>
<th>Curso</th>
<th>Créditos</th>
<th>N Estudiantes</th>
</tr>
<tr>
<td>IMT1101</td>
<td>10</td>
<td>42</td>
</tr>
<tr>
<td>OFG1100</td>
<td>5</td>
<td>25</td>
</tr>
</table>


El método `find()` devuelve el primer elemento encontrado. Para escanear todo el documento y extraer más de un elemento (por ejemplo, si en la página existieran varias tablas), se utiliza el método `find_all()`.

Ver: https://beautiful-soup-4.readthedocs.io/en/latest/index.html?highlight=find#find

Una vez extraída la tabla, usamos `find_all()` para encontrar todas las filas de la tabla (`'tr'`: table rows), e iteramos sobre las filas. Para cada fila, encontramos todos los campos de datos (`'td'`: table data), y vamos agregando los datos a un DataFrame especialmente creado para este fin.

In [16]:
import pandas as pd

#creamos un DataFrame vacío con la estructura de columnas deseada. Lo completaremos con datos extraidos de la tabla.
df=pd.DataFrame(columns=['Curso','Creditos','nEstudiantes'])

#Buscar la tabla con datos
table=soup.find('table')

#Buscar todas las filas de la tabla
rows=table.find_all('tr')

#Iteramos para cada fila (excepto la primera que corresponde a los encabezados)

for row in rows[1::]:
    
    #Buscar todos los campos de datos o columnas de la fila
    cols = row.find_all("td")
    
    #Para cada columna, extraigo el texto
    col_text=[c.text for c in cols]
    
    #Agrego los datos como una nueva fila en el dataframe df
    df=df.append({'Curso':col_text[0],'Creditos':col_text[1],'nEstudiantes':col_text[2]},ignore_index=True)
df

<ipython-input-16-a1ca27efb750>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'Curso':col_text[0],'Creditos':col_text[1],'nEstudiantes':col_text[2]},ignore_index=True)
<ipython-input-16-a1ca27efb750>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'Curso':col_text[0],'Creditos':col_text[1],'nEstudiantes':col_text[2]},ignore_index=True)


,Curso,Creditos,nEstudiantes
0,IMT1101,10,42
1,OFG1100,5,25
